In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
import cv2
import numpy as np

In [2]:
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.models import Model

Using TensorFlow backend.


In [5]:
image2 = cv2.imread('image2.jpg')
image2 = np.array(image2,dtype='float32')
image2 = np.expand_dims(image2,axis=0)
image2 = preprocess_input(image2)

In [6]:
im1 = cv2.imread('../features_videos_inv3/Chatzistefanou_Aris_Debtocracy/shots/531960/531960.png')
im1 = np.array(im1,dtype='float32')
im1 = np.expand_dims(im1,axis=0)
im1 = preprocess_input(im1)

im2 = cv2.imread('../features_videos_inv3/Chatzistefanou_Aris_Debtocracy/shots/444040/444040.png')
im2 = np.array(im2,dtype='float32')
im2 = np.expand_dims(im2,axis=0)
im2 = preprocess_input(im2)

im3 = cv2.imread('../features_videos_inv3/Chatzistefanou_Aris_Debtocracy/shots/531960/531962.png')
im3 = np.array(im3,dtype='float32')
im3 = np.expand_dims(im3,axis=0)
im3 = preprocess_input(im3)

In [8]:
model = InceptionV3(weights='imagenet',input_shape=(224,224,3))
model = Model(inputs=model.input,output=model.get_layer('avg_pool').output)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("av...)`
  
